In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("shortest path assignment").setMaster("local"))

In [2]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

In [3]:
def myStep(item): 
    v, d, nv = item[0], item[1][0], item[1][1]; 
    path = d + (v,)
    return (nv, (path), v)

In [8]:
def findShortPath(nodes = None, src = None, dst = None, nIteration = 100):
    
    if nodes == None or src == None or dst == None or nIteration <= 0:
        return (None, None)
    
    fwd_nodes = nodes.map(lambda e: (e[1], e[0])).persist() # swap
    dist = sc.parallelize([(src,(0,))])
    
    while nIteration != 0:  
        cand = dist.join(fwd_nodes).persist()
        new_dist = cand.map(myStep).persist()
        path = new_dist.lookup(dst)
        if len(path) > 0:
            #print path
            break
        dist = new_dist
        nIteration -= 1
        #print nIteration
    shortPath = []
    for p in path:
        delSrc = p[1:]
        shortPath.append(delSrc + (dst,))
    return (nIteration, shortPath)

In [9]:
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()

In [10]:
(nInteration, path) = findShortPath(edges, 12, 34, 11)

In [11]:
out = []
for p in path[0]:
    out.append(str(p))
outCvs = ",".join(out)
print outCvs

12,422,53,52,107,20,23,274,34
